# Turning on the Jupyter Auto-Completor

In [1]:
%config Completer.use_jedi = False

# Importing the required libraries

In [2]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import warnings
warnings.filterwarnings('ignore')

# Getting the IMDbMovieDataset

In [3]:
movies = pd.read_csv("Datasets/IMDBMovieDataset.csv")
movies.head()

,Movie ID,Movie Link,Movie Poster,Movie Name,Movie Year,Movie Genre,Movie Rating,Movie Description,Movie Director,Movie Actors,Movie Actors Image
0,tt1375666,https://www.imdb.com//title/tt1375666/?ref_=ad...,https://m.media-amazon.com/images/M/MV5BYjZjOW...,Inception,2010,"Action, Adventure, Sci-Fi",8.8,A thief who steals corporate secrets through t...,Christopher Nolan,"['Leonardo DiCaprio', 'Joseph Gordon-Levitt', ...",['https://m.media-amazon.com/images/M/MV5BMjI0...
1,tt0816692,https://www.imdb.com//title/tt0816692/?ref_=ad...,https://m.media-amazon.com/images/M/MV5BZjdkOT...,Interstellar,2014,"Adventure, Drama, Sci-Fi",8.6,A team of explorers travel through a wormhole ...,Christopher Nolan,"['Matthew McConaughey', 'Anne Hathaway', 'Jess...",['https://m.media-amazon.com/images/M/MV5BMTg0...
2,tt1345836,https://www.imdb.com//title/tt1345836/?ref_=ad...,https://m.media-amazon.com/images/M/MV5BMTk4OD...,The Dark Knight Rises,2012,"Action, Drama",8.4,Eight years after the Joker's reign of anarchy...,Christopher Nolan,"['Christian Bale', 'Tom Hardy', 'Anne Hathaway...",['https://m.media-amazon.com/images/M/MV5BMTkx...
3,tt1853728,https://www.imdb.com//title/tt1853728/?ref_=ad...,https://m.media-amazon.com/images/M/MV5BMjIyNT...,Django Unchained,2012,"Drama, Western",8.4,"With the help of a German bounty-hunter, a fre...",Quentin Tarantino,"['Jamie Foxx', 'Christoph Waltz', 'Leonardo Di...",['https://m.media-amazon.com/images/M/MV5BMTky...
4,tt0848228,https://www.imdb.com//title/tt0848228/?ref_=ad...,https://m.media-amazon.com/images/M/MV5BOTgxZG...,The Avengers,2012,"Action, Adventure, Sci-Fi",8.0,Earth's mightiest heroes must come together an...,Joss Whedon,"['Robert Downey Jr.', 'Chris Evans', 'Scarlett...",['https://m.media-amazon.com/images/M/MV5BNzg1...


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5070 entries, 0 to 5069
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Movie ID            5070 non-null   object 
 1   Movie Link          5070 non-null   object 
 2   Movie Poster        5070 non-null   object 
 3   Movie Name          5070 non-null   object 
 4   Movie Year          5070 non-null   int64  
 5   Movie Genre         5070 non-null   object 
 6   Movie Rating        5070 non-null   float64
 7   Movie Description   5070 non-null   object 
 8   Movie Director      5070 non-null   object 
 9   Movie Actors        5070 non-null   object 
 10  Movie Actors Image  5070 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 435.8+ KB


In [5]:
movies.describe()

,Movie Year,Movie Rating
count,5070.000000,5070.000000
mean,2015.341026,6.269270
std,3.492838,1.031823
min,1980.000000,1.100000
25%,2013.000000,5.700000
50%,2015.000000,6.300000
75%,2018.000000,7.000000
max,2022.000000,9.500000


In [6]:
movies.duplicated().sum()

36

In [7]:
movies.drop_duplicates(inplace=True)

In [8]:
movies.head()

,Movie ID,Movie Link,Movie Poster,Movie Name,Movie Year,Movie Genre,Movie Rating,Movie Description,Movie Director,Movie Actors,Movie Actors Image
0,tt1375666,https://www.imdb.com//title/tt1375666/?ref_=ad...,https://m.media-amazon.com/images/M/MV5BYjZjOW...,Inception,2010,"Action, Adventure, Sci-Fi",8.8,A thief who steals corporate secrets through t...,Christopher Nolan,"['Leonardo DiCaprio', 'Joseph Gordon-Levitt', ...",['https://m.media-amazon.com/images/M/MV5BMjI0...
1,tt0816692,https://www.imdb.com//title/tt0816692/?ref_=ad...,https://m.media-amazon.com/images/M/MV5BZjdkOT...,Interstellar,2014,"Adventure, Drama, Sci-Fi",8.6,A team of explorers travel through a wormhole ...,Christopher Nolan,"['Matthew McConaughey', 'Anne Hathaway', 'Jess...",['https://m.media-amazon.com/images/M/MV5BMTg0...
2,tt1345836,https://www.imdb.com//title/tt1345836/?ref_=ad...,https://m.media-amazon.com/images/M/MV5BMTk4OD...,The Dark Knight Rises,2012,"Action, Drama",8.4,Eight years after the Joker's reign of anarchy...,Christopher Nolan,"['Christian Bale', 'Tom Hardy', 'Anne Hathaway...",['https://m.media-amazon.com/images/M/MV5BMTkx...
3,tt1853728,https://www.imdb.com//title/tt1853728/?ref_=ad...,https://m.media-amazon.com/images/M/MV5BMjIyNT...,Django Unchained,2012,"Drama, Western",8.4,"With the help of a German bounty-hunter, a fre...",Quentin Tarantino,"['Jamie Foxx', 'Christoph Waltz', 'Leonardo Di...",['https://m.media-amazon.com/images/M/MV5BMTky...
4,tt0848228,https://www.imdb.com//title/tt0848228/?ref_=ad...,https://m.media-amazon.com/images/M/MV5BOTgxZG...,The Avengers,2012,"Action, Adventure, Sci-Fi",8.0,Earth's mightiest heroes must come together an...,Joss Whedon,"['Robert Downey Jr.', 'Chris Evans', 'Scarlett...",['https://m.media-amazon.com/images/M/MV5BNzg1...


In [9]:
movies.duplicated().sum()

0

# Finding out which columns are important for Machine Learning Model

## Movie ID
## Movie Name
## Movie Genre
## Movie Description
## Movie Director
## Movie Actors

In [10]:
movies_df = movies[['Movie ID', 'Movie Name', 'Movie Genre', 'Movie Description', 'Movie Director', 'Movie Actors']]
movies_df

,Movie ID,Movie Name,Movie Genre,Movie Description,Movie Director,Movie Actors
0,tt1375666,Inception,"Action, Adventure, Sci-Fi",A thief who steals corporate secrets through t...,Christopher Nolan,"['Leonardo DiCaprio', 'Joseph Gordon-Levitt', ..."
1,tt0816692,Interstellar,"Adventure, Drama, Sci-Fi",A team of explorers travel through a wormhole ...,Christopher Nolan,"['Matthew McConaughey', 'Anne Hathaway', 'Jess..."
2,tt1345836,The Dark Knight Rises,"Action, Drama",Eight years after the Joker's reign of anarchy...,Christopher Nolan,"['Christian Bale', 'Tom Hardy', 'Anne Hathaway..."
3,tt1853728,Django Unchained,"Drama, Western","With the help of a German bounty-hunter, a fre...",Quentin Tarantino,"['Jamie Foxx', 'Christoph Waltz', 'Leonardo Di..."
4,tt0848228,The Avengers,"Action, Adventure, Sci-Fi",Earth's mightiest heroes must come together an...,Joss Whedon,"['Robert Downey Jr.', 'Chris Evans', 'Scarlett..."
...,...,...,...,...,...,...
5065,tt1133985,Green Lantern,"Action, Adventure, Sci-Fi",Reckless test pilot Hal Jordan is granted an a...,Martin Campbell,"['Ryan Reynolds', 'Blake Lively', 'Peter Sarsg..."
5066,tt0086393,Superman III,"Action, Adventure, Comedy",Synthetic kryptonite laced with tar splits Sup...,Richard Lester,"['Christopher Reeve', 'Richard Pryor', 'Margot..."
5067,tt0348150,Superman Returns,"Action, Adventure, Sci-Fi",Superman returns to Earth after spending five ...,Bryan Singer,"['Brandon Routh', 'Kevin Spacey', 'Kate Boswor..."
5068,tt0081573,Superman II,"Action, Adventure, Sci-Fi",Superman agrees to sacrifice his powers to sta...,Richard Lester,"['Gene Hackman', 'Christopher Reeve', 'Margot ..."


# Performing the Data Preprocessing

In [11]:
movies_df.iloc[0]

Movie ID                                                     tt1375666
Movie Name                                                   Inception
Movie Genre                                  Action, Adventure, Sci-Fi
Movie Description    A thief who steals corporate secrets through t...
Movie Director                                       Christopher Nolan
Movie Actors         ['Leonardo DiCaprio', 'Joseph Gordon-Levitt', ...
Name: 0, dtype: object

In [12]:
movies['Movie Genre'][0]

'Action, Adventure, Sci-Fi'

In [13]:
movies_df['Movie Description'].apply(lambda x:x.split())

0       [A, thief, who, steals, corporate, secrets, th...
1       [A, team, of, explorers, travel, through, a, w...
2       [Eight, years, after, the, Joker's, reign, of,...
3       [With, the, help, of, a, German, bounty-hunter...
4       [Earth's, mightiest, heroes, must, come, toget...
                              ...                        
5065    [Reckless, test, pilot, Hal, Jordan, is, grant...
5066    [Synthetic, kryptonite, laced, with, tar, spli...
5067    [Superman, returns, to, Earth, after, spending...
5068    [Superman, agrees, to, sacrifice, his, powers,...
5069    [Superman, agrees, to, sacrifice, his, powers,...
Name: Movie Description, Length: 5034, dtype: object

In [14]:
movies_df['Movie Description'] = movies_df['Movie Description'].apply(lambda x:x.split())

In [15]:
movies_df['Movie Genre'] = movies_df['Movie Genre'].apply(lambda x:x.split())

In [16]:
movies_df.head()

,Movie ID,Movie Name,Movie Genre,Movie Description,Movie Director,Movie Actors
0,tt1375666,Inception,"[Action,, Adventure,, Sci-Fi]","[A, thief, who, steals, corporate, secrets, th...",Christopher Nolan,"['Leonardo DiCaprio', 'Joseph Gordon-Levitt', ..."
1,tt0816692,Interstellar,"[Adventure,, Drama,, Sci-Fi]","[A, team, of, explorers, travel, through, a, w...",Christopher Nolan,"['Matthew McConaughey', 'Anne Hathaway', 'Jess..."
2,tt1345836,The Dark Knight Rises,"[Action,, Drama]","[Eight, years, after, the, Joker's, reign, of,...",Christopher Nolan,"['Christian Bale', 'Tom Hardy', 'Anne Hathaway..."
3,tt1853728,Django Unchained,"[Drama,, Western]","[With, the, help, of, a, German, bounty-hunter...",Quentin Tarantino,"['Jamie Foxx', 'Christoph Waltz', 'Leonardo Di..."
4,tt0848228,The Avengers,"[Action,, Adventure,, Sci-Fi]","[Earth's, mightiest, heroes, must, come, toget...",Joss Whedon,"['Robert Downey Jr.', 'Chris Evans', 'Scarlett..."


In [17]:
movies_df['Movie Genre'].apply(lambda x:[i.replace(" ", "") for i in x])

0        [Action,, Adventure,, Sci-Fi]
1         [Adventure,, Drama,, Sci-Fi]
2                     [Action,, Drama]
3                    [Drama,, Western]
4        [Action,, Adventure,, Sci-Fi]
                     ...              
5065     [Action,, Adventure,, Sci-Fi]
5066     [Action,, Adventure,, Comedy]
5067     [Action,, Adventure,, Sci-Fi]
5068     [Action,, Adventure,, Sci-Fi]
5069    [Action,, Adventure,, Romance]
Name: Movie Genre, Length: 5034, dtype: object

In [18]:
movies_df['Movie Genre'].apply(lambda x:[i.replace(",", "") for i in x])

0        [Action, Adventure, Sci-Fi]
1         [Adventure, Drama, Sci-Fi]
2                    [Action, Drama]
3                   [Drama, Western]
4        [Action, Adventure, Sci-Fi]
                    ...             
5065     [Action, Adventure, Sci-Fi]
5066     [Action, Adventure, Comedy]
5067     [Action, Adventure, Sci-Fi]
5068     [Action, Adventure, Sci-Fi]
5069    [Action, Adventure, Romance]
Name: Movie Genre, Length: 5034, dtype: object

In [19]:
movies_df['Movie Genre'] = movies_df['Movie Genre'].apply(lambda x:[i.replace(" ", "") for i in x])

In [20]:
movies_df['Movie Genre'] = movies_df['Movie Genre'].apply(lambda x:[i.replace(",", "") for i in x])

In [21]:
movies_df.head()

,Movie ID,Movie Name,Movie Genre,Movie Description,Movie Director,Movie Actors
0,tt1375666,Inception,"[Action, Adventure, Sci-Fi]","[A, thief, who, steals, corporate, secrets, th...",Christopher Nolan,"['Leonardo DiCaprio', 'Joseph Gordon-Levitt', ..."
1,tt0816692,Interstellar,"[Adventure, Drama, Sci-Fi]","[A, team, of, explorers, travel, through, a, w...",Christopher Nolan,"['Matthew McConaughey', 'Anne Hathaway', 'Jess..."
2,tt1345836,The Dark Knight Rises,"[Action, Drama]","[Eight, years, after, the, Joker's, reign, of,...",Christopher Nolan,"['Christian Bale', 'Tom Hardy', 'Anne Hathaway..."
3,tt1853728,Django Unchained,"[Drama, Western]","[With, the, help, of, a, German, bounty-hunter...",Quentin Tarantino,"['Jamie Foxx', 'Christoph Waltz', 'Leonardo Di..."
4,tt0848228,The Avengers,"[Action, Adventure, Sci-Fi]","[Earth's, mightiest, heroes, must, come, toget...",Joss Whedon,"['Robert Downey Jr.', 'Chris Evans', 'Scarlett..."


In [22]:
def formatterA(text):
    lst = list(text[2:-2].split("', '"))
    return lst

In [23]:
movies_df['Movie Actors'].apply(formatterA)

0       [Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...
1       [Matthew McConaughey, Anne Hathaway, Jessica C...
2       [Christian Bale, Tom Hardy, Anne Hathaway, Gar...
3       [Jamie Foxx, Christoph Waltz, Leonardo DiCapri...
4       [Robert Downey Jr., Chris Evans, Scarlett Joha...
                              ...                        
5065    [Ryan Reynolds, Blake Lively, Peter Sarsgaard,...
5066    [Christopher Reeve, Richard Pryor, Margot Kidd...
5067    [Brandon Routh, Kevin Spacey, Kate Bosworth, J...
5068    [Gene Hackman, Christopher Reeve, Margot Kidde...
5069    [Gene Hackman, Christopher Reeve, Marlon Brand...
Name: Movie Actors, Length: 5034, dtype: object

In [24]:
movies_df['Movie Actors'] = movies_df['Movie Actors'].apply(formatterA)

In [25]:
movies_df.head()

,Movie ID,Movie Name,Movie Genre,Movie Description,Movie Director,Movie Actors
0,tt1375666,Inception,"[Action, Adventure, Sci-Fi]","[A, thief, who, steals, corporate, secrets, th...",Christopher Nolan,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli..."
1,tt0816692,Interstellar,"[Adventure, Drama, Sci-Fi]","[A, team, of, explorers, travel, through, a, w...",Christopher Nolan,"[Matthew McConaughey, Anne Hathaway, Jessica C..."
2,tt1345836,The Dark Knight Rises,"[Action, Drama]","[Eight, years, after, the, Joker's, reign, of,...",Christopher Nolan,"[Christian Bale, Tom Hardy, Anne Hathaway, Gar..."
3,tt1853728,Django Unchained,"[Drama, Western]","[With, the, help, of, a, German, bounty-hunter...",Quentin Tarantino,"[Jamie Foxx, Christoph Waltz, Leonardo DiCapri..."
4,tt0848228,The Avengers,"[Action, Adventure, Sci-Fi]","[Earth's, mightiest, heroes, must, come, toget...",Joss Whedon,"[Robert Downey Jr., Chris Evans, Scarlett Joha..."


In [26]:
movies_df['Movie Actors'].apply(lambda x:[i.replace(" ", "") for i in x])

0       [LeonardoDiCaprio, JosephGordon-Levitt, Elliot...
1       [MatthewMcConaughey, AnneHathaway, JessicaChas...
2       [ChristianBale, TomHardy, AnneHathaway, GaryOl...
3       [JamieFoxx, ChristophWaltz, LeonardoDiCaprio, ...
4       [RobertDowneyJr., ChrisEvans, ScarlettJohansso...
                              ...                        
5065    [RyanReynolds, BlakeLively, PeterSarsgaard, Ma...
5066    [ChristopherReeve, RichardPryor, MargotKidder,...
5067    [BrandonRouth, KevinSpacey, KateBosworth, Jame...
5068    [GeneHackman, ChristopherReeve, MargotKidder, ...
5069    [GeneHackman, ChristopherReeve, MarlonBrando, ...
Name: Movie Actors, Length: 5034, dtype: object

In [27]:
movies_df['Movie Actors'] = movies_df['Movie Actors'].apply(lambda x:[i.replace(" ", "") for i in x])
movies_df.head()

,Movie ID,Movie Name,Movie Genre,Movie Description,Movie Director,Movie Actors
0,tt1375666,Inception,"[Action, Adventure, Sci-Fi]","[A, thief, who, steals, corporate, secrets, th...",Christopher Nolan,"[LeonardoDiCaprio, JosephGordon-Levitt, Elliot..."
1,tt0816692,Interstellar,"[Adventure, Drama, Sci-Fi]","[A, team, of, explorers, travel, through, a, w...",Christopher Nolan,"[MatthewMcConaughey, AnneHathaway, JessicaChas..."
2,tt1345836,The Dark Knight Rises,"[Action, Drama]","[Eight, years, after, the, Joker's, reign, of,...",Christopher Nolan,"[ChristianBale, TomHardy, AnneHathaway, GaryOl..."
3,tt1853728,Django Unchained,"[Drama, Western]","[With, the, help, of, a, German, bounty-hunter...",Quentin Tarantino,"[JamieFoxx, ChristophWaltz, LeonardoDiCaprio, ..."
4,tt0848228,The Avengers,"[Action, Adventure, Sci-Fi]","[Earth's, mightiest, heroes, must, come, toget...",Joss Whedon,"[RobertDowneyJr., ChrisEvans, ScarlettJohansso..."


In [28]:
movies_df['Movie Director'][0].split()

['Christopher', 'Nolan']

In [29]:
def formatterD(text):
    lst = text.split()
    text = ''.join(lst)
    return text.split()

In [30]:
formatterD(movies_df['Movie Director'][1])

['ChristopherNolan']

In [31]:
movies_df['Movie Director'].apply(formatterD)

0       [ChristopherNolan]
1       [ChristopherNolan]
2       [ChristopherNolan]
3       [QuentinTarantino]
4             [JossWhedon]
               ...        
5065      [MartinCampbell]
5066       [RichardLester]
5067         [BryanSinger]
5068       [RichardLester]
5069       [RichardDonner]
Name: Movie Director, Length: 5034, dtype: object

In [32]:
movies_df['Movie Director'] = movies_df['Movie Director'].apply(formatterD)

In [33]:
movies_df.head()

,Movie ID,Movie Name,Movie Genre,Movie Description,Movie Director,Movie Actors
0,tt1375666,Inception,"[Action, Adventure, Sci-Fi]","[A, thief, who, steals, corporate, secrets, th...",[ChristopherNolan],"[LeonardoDiCaprio, JosephGordon-Levitt, Elliot..."
1,tt0816692,Interstellar,"[Adventure, Drama, Sci-Fi]","[A, team, of, explorers, travel, through, a, w...",[ChristopherNolan],"[MatthewMcConaughey, AnneHathaway, JessicaChas..."
2,tt1345836,The Dark Knight Rises,"[Action, Drama]","[Eight, years, after, the, Joker's, reign, of,...",[ChristopherNolan],"[ChristianBale, TomHardy, AnneHathaway, GaryOl..."
3,tt1853728,Django Unchained,"[Drama, Western]","[With, the, help, of, a, German, bounty-hunter...",[QuentinTarantino],"[JamieFoxx, ChristophWaltz, LeonardoDiCaprio, ..."
4,tt0848228,The Avengers,"[Action, Adventure, Sci-Fi]","[Earth's, mightiest, heroes, must, come, toget...",[JossWhedon],"[RobertDowneyJr., ChrisEvans, ScarlettJohansso..."


In [34]:
movies_df['Tags'] = movies_df['Movie Genre'] + movies_df['Movie Description'] + movies_df['Movie Director'] + movies_df['Movie Actors']

In [35]:
movies_df.head()

,Movie ID,Movie Name,Movie Genre,Movie Description,Movie Director,Movie Actors,Tags
0,tt1375666,Inception,"[Action, Adventure, Sci-Fi]","[A, thief, who, steals, corporate, secrets, th...",[ChristopherNolan],"[LeonardoDiCaprio, JosephGordon-Levitt, Elliot...","[Action, Adventure, Sci-Fi, A, thief, who, ste..."
1,tt0816692,Interstellar,"[Adventure, Drama, Sci-Fi]","[A, team, of, explorers, travel, through, a, w...",[ChristopherNolan],"[MatthewMcConaughey, AnneHathaway, JessicaChas...","[Adventure, Drama, Sci-Fi, A, team, of, explor..."
2,tt1345836,The Dark Knight Rises,"[Action, Drama]","[Eight, years, after, the, Joker's, reign, of,...",[ChristopherNolan],"[ChristianBale, TomHardy, AnneHathaway, GaryOl...","[Action, Drama, Eight, years, after, the, Joke..."
3,tt1853728,Django Unchained,"[Drama, Western]","[With, the, help, of, a, German, bounty-hunter...",[QuentinTarantino],"[JamieFoxx, ChristophWaltz, LeonardoDiCaprio, ...","[Drama, Western, With, the, help, of, a, Germa..."
4,tt0848228,The Avengers,"[Action, Adventure, Sci-Fi]","[Earth's, mightiest, heroes, must, come, toget...",[JossWhedon],"[RobertDowneyJr., ChrisEvans, ScarlettJohansso...","[Action, Adventure, Sci-Fi, Earth's, mightiest..."


In [36]:
print(movies_df['Tags'][0])

['Action', 'Adventure', 'Sci-Fi', 'A', 'thief', 'who', 'steals', 'corporate', 'secrets', 'through', 'the', 'use', 'of', 'dream-sharing', 'technology', 'is', 'given', 'the', 'inverse', 'task', 'of', 'planting', 'an', 'idea', 'into', 'the', 'mind', 'of', 'a', 'C.E.O.,', 'but', 'his', 'tragic', 'past', 'may', 'doom', 'the', 'project', 'and', 'his', 'team', 'to', 'disaster.', 'ChristopherNolan', 'LeonardoDiCaprio', 'JosephGordon-Levitt', 'ElliotPage', 'KenWatanabe', 'TomHardy']


In [37]:
movies_df = movies_df[['Movie ID', 'Movie Name', 'Tags']]

In [38]:
movies_df.head()

,Movie ID,Movie Name,Tags
0,tt1375666,Inception,"[Action, Adventure, Sci-Fi, A, thief, who, ste..."
1,tt0816692,Interstellar,"[Adventure, Drama, Sci-Fi, A, team, of, explor..."
2,tt1345836,The Dark Knight Rises,"[Action, Drama, Eight, years, after, the, Joke..."
3,tt1853728,Django Unchained,"[Drama, Western, With, the, help, of, a, Germa..."
4,tt0848228,The Avengers,"[Action, Adventure, Sci-Fi, Earth's, mightiest..."


In [39]:
movies_df['Tags'].apply(lambda x: " ".join(x))

0       Action Adventure Sci-Fi A thief who steals cor...
1       Adventure Drama Sci-Fi A team of explorers tra...
2       Action Drama Eight years after the Joker's rei...
3       Drama Western With the help of a German bounty...
4       Action Adventure Sci-Fi Earth's mightiest hero...
                              ...                        
5065    Action Adventure Sci-Fi Reckless test pilot Ha...
5066    Action Adventure Comedy Synthetic kryptonite l...
5067    Action Adventure Sci-Fi Superman returns to Ea...
5068    Action Adventure Sci-Fi Superman agrees to sac...
5069    Action Adventure Romance Superman agrees to sa...
Name: Tags, Length: 5034, dtype: object

In [40]:
movies_df['Tags'] = movies_df['Tags'].apply(lambda x: " ".join(x))
movies_df.head()

,Movie ID,Movie Name,Tags
0,tt1375666,Inception,Action Adventure Sci-Fi A thief who steals cor...
1,tt0816692,Interstellar,Adventure Drama Sci-Fi A team of explorers tra...
2,tt1345836,The Dark Knight Rises,Action Drama Eight years after the Joker's rei...
3,tt1853728,Django Unchained,Drama Western With the help of a German bounty...
4,tt0848228,The Avengers,Action Adventure Sci-Fi Earth's mightiest hero...


In [41]:
movies_df['Tags'][0]

'Action Adventure Sci-Fi A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster. ChristopherNolan LeonardoDiCaprio JosephGordon-Levitt ElliotPage KenWatanabe TomHardy'

In [42]:
movies_df['Tags'] = movies_df['Tags'].apply(lambda x: x.lower())
movies_df.head()

,Movie ID,Movie Name,Tags
0,tt1375666,Inception,action adventure sci-fi a thief who steals cor...
1,tt0816692,Interstellar,adventure drama sci-fi a team of explorers tra...
2,tt1345836,The Dark Knight Rises,action drama eight years after the joker's rei...
3,tt1853728,Django Unchained,drama western with the help of a german bounty...
4,tt0848228,The Avengers,action adventure sci-fi earth's mightiest hero...


# Removing the Stop Words and Stemming

In [43]:
ps = PorterStemmer()

In [44]:
lst_stopwords = stopwords.words('english')

In [45]:
def removeStopwords(text):
    res = []
    for i in text.split():
        if i not in lst_stopwords:
            res.append(i)
    res = ' '.join(res)
    return res

In [46]:
removeStopwords(movies_df['Tags'][0])

'action adventure sci-fi thief steals corporate secrets use dream-sharing technology given inverse task planting idea mind c.e.o., tragic past may doom project team disaster. christophernolan leonardodicaprio josephgordon-levitt elliotpage kenwatanabe tomhardy'

In [47]:
def stemText(text):
    res = []
    for i in text.split():
        res.append(ps.stem(i))
    return res

In [48]:
print(stemText(removeStopwords(movies_df['Tags'][0])))

['action', 'adventur', 'sci-fi', 'thief', 'steal', 'corpor', 'secret', 'use', 'dream-shar', 'technolog', 'given', 'invers', 'task', 'plant', 'idea', 'mind', 'c.e.o.,', 'tragic', 'past', 'may', 'doom', 'project', 'team', 'disaster.', 'christophernolan', 'leonardodicaprio', 'josephgordon-levitt', 'elliotpag', 'kenwatanab', 'tomhardi']


In [49]:
def preprocessingData(text):
    res = []
    for i in text.split():
        if i not in lst_stopwords:
            i = ps.stem(i)
            res.append(i)
    res = ' '.join(res)
    return res

In [50]:
preprocessingData(movies_df['Tags'][0])

'action adventur sci-fi thief steal corpor secret use dream-shar technolog given invers task plant idea mind c.e.o., tragic past may doom project team disaster. christophernolan leonardodicaprio josephgordon-levitt elliotpag kenwatanab tomhardi'

In [51]:
movies_df['Tags'].apply(preprocessingData)

0       action adventur sci-fi thief steal corpor secr...
1       adventur drama sci-fi team explor travel wormh...
2       action drama eight year joker' reign anarchy, ...
3       drama western help german bounty-hunter, freed...
4       action adventur sci-fi earth' mightiest hero m...
                              ...                        
5065    action adventur sci-fi reckless test pilot hal...
5066    action adventur comedi synthet kryptonit lace ...
5067    action adventur sci-fi superman return earth s...
5068    action adventur sci-fi superman agre sacrific ...
5069    action adventur romanc superman agre sacrific ...
Name: Tags, Length: 5034, dtype: object

In [52]:
movies_df['Tags'] = movies_df['Tags'].apply(preprocessingData)

In [53]:
cv = CountVectorizer(max_features=4000, stop_words='english')

In [54]:
vectors = cv.fit_transform(movies_df['Tags']).toarray()

In [55]:
cv.fit_transform(movies_df['Tags']).toarray().shape

(5034, 4000)

In [56]:
# print(cv.get_feature_names())

# Generating intermediate Cosine distance

In [57]:
similarity = cosine_similarity(vectors)

In [58]:
similarity.shape

(5034, 5034)

In [59]:
similarity

array([[1.        , 0.22518867, 0.1979519 , ..., 0.16208818, 0.17507524,
        0.09007547],
       [0.22518867, 1.        , 0.15512631, ..., 0.21170245, 0.17149859,
        0.05882353],
       [0.1979519 , 0.15512631, 1.        , ..., 0.09304842, 0.05025189,
        0.05170877],
       ...,
       [0.16208818, 0.21170245, 0.09304842, ..., 1.        , 0.3086067 ,
        0.21170245],
       [0.17507524, 0.17149859, 0.05025189, ..., 0.3086067 , 1.        ,
        0.91465912],
       [0.09007547, 0.05882353, 0.05170877, ..., 0.21170245, 0.91465912,
        1.        ]])

# Building the Recommendation System

In [60]:
movies_df.head()

,Movie ID,Movie Name,Tags
0,tt1375666,Inception,action adventur sci-fi thief steal corpor secr...
1,tt0816692,Interstellar,adventur drama sci-fi team explor travel wormh...
2,tt1345836,The Dark Knight Rises,"action drama eight year joker' reign anarchy, ..."
3,tt1853728,Django Unchained,"drama western help german bounty-hunter, freed..."
4,tt0848228,The Avengers,action adventur sci-fi earth' mightiest hero m...


In [61]:
def recommendedMovies(movie):
    movie_index = movies_df[movies_df['Movie Name'] == movie].index[0]
    distances = similarity[movie_index]
    recommended_movies_index = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_movies = []
    for i in recommended_movies_index:
        recommended_movies.append(movies_df.iloc[i[0]]['Movie Name'])
    return recommended_movies

In [62]:
def recommendedMoviesID(movie):
    movie_index = movies_df[movies_df['Movie Name'] == movie].index[0]
    distances = similarity[movie_index]
    recommended_movies_index = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_movies_id = []
    for i in recommended_movies_index:
        recommended_movies_id.append(i[0])
    return recommended_movies_id

In [63]:
recommendedMovies('Spider-Man: No Way Home')

['Spider-Man: Far from Home',
 'Spider-Man: Homecoming',
 'Iron Man 2',
 'The Amazing Spider-Man',
 'Pan']

In [64]:
recommendedMoviesID('Spider-Man: No Way Home')

[131, 66, 28, 63, 1151]

In [65]:
movies_df['Movie Name'][63]

'The Amazing Spider-Man'

# Finally generating CSV file of result for faster processing

In [66]:
recommended_movies = movies_df[['Movie ID', 'Movie Name']]
recommended_movies.head()

,Movie ID,Movie Name
0,tt1375666,Inception
1,tt0816692,Interstellar
2,tt1345836,The Dark Knight Rises
3,tt1853728,Django Unchained
4,tt0848228,The Avengers


In [67]:
movies_df.shape

(5034, 3)

In [68]:
movies_df['Movie Name'][2580:2600]

2580                       Ying
2581           The Sea of Trees
2582                  Marmaduke
2584            La Belle Époque
2585                  Detention
2586                    October
2587             Mirai no Mirai
2588         Satyameva Jayate 2
2589                  Cold Skin
2590               Mary Shelley
2591    Shaadi Mein Zaroor Aana
2592    Kimi no suizô o tabetai
2593    Des hommes et des dieux
2594                Section 375
2595                     Mine 9
2596            Very Good Girls
2597     Dhuruvangal Pathinaaru
2598                      Gotti
2599                  Happy End
2600           Girl Most Likely
Name: Movie Name, dtype: object

In [69]:
movies_df.reset_index(inplace=True)

In [70]:
movies_df['Movie Name'][2580:2600]

2580                       Ying
2581           The Sea of Trees
2582                  Marmaduke
2583            La Belle Époque
2584                  Detention
2585                    October
2586             Mirai no Mirai
2587         Satyameva Jayate 2
2588                  Cold Skin
2589               Mary Shelley
2590    Shaadi Mein Zaroor Aana
2591    Kimi no suizô o tabetai
2592    Des hommes et des dieux
2593                Section 375
2594                     Mine 9
2595            Very Good Girls
2596     Dhuruvangal Pathinaaru
2597                      Gotti
2598                  Happy End
2599           Girl Most Likely
Name: Movie Name, dtype: object

In [71]:
"""for i in range(0, 5034, 1):
    movie_name = movies_df['Movie Name'][i]
    print(movie_name)"""

"for i in range(0, 5034, 1):\n    movie_name = movies_df['Movie Name'][i]\n    print(movie_name)"

In [72]:
movies_df['Movie Name'].apply(recommendedMovies)

0       [Spectral, Alita: Battle Angel, Godzilla, Blac...
1       [Spectral, The Martian, Stake Land, Sound of M...
2       [Batman Begins, The Dark Knight, 7500, Batman ...
3       [The Legend of Tarzan, Water for Elephants, Af...
4       [Avengers: Age of Ultron, Avengers: Infinity W...
                              ...                        
5029    [Captain Marvel, I Am Number Four, Ra.One, Bey...
5030    [Batman v Superman: Dawn of Justice, Bang Bang...
5031    [Capsule, Batman v Superman: Dawn of Justice, ...
5032    [Superman II: The Richard Donner Cut, Robot Ov...
5033    [Superman II, Injustice, Comment je suis deven...
Name: Movie Name, Length: 5034, dtype: object

In [73]:
recommended_movies['Reccomendation'] = movies_df['Movie Name'].apply(recommendedMovies)

In [74]:
recommended_movies['ReccomendationID'] = movies_df['Movie Name'].apply(recommendedMoviesID)

In [75]:
recommended_movies.head()

,Movie ID,Movie Name,Reccomendation,ReccomendationID
0,tt1375666,Inception,"[Spectral, Alita: Battle Angel, Godzilla, Blac...","[1209, 289, 166, 197, 41]"
1,tt0816692,Interstellar,"[Spectral, The Martian, Stake Land, Sound of M...","[1209, 19, 1515, 2163, 2912]"
2,tt1345836,The Dark Knight Rises,"[Batman Begins, The Dark Knight, 7500, Batman ...","[5025, 5022, 1902, 2275, 4712]"
3,tt1853728,Django Unchained,"[The Legend of Tarzan, Water for Elephants, Af...","[483, 731, 3852, 3154, 4740]"
4,tt0848228,The Avengers,"[Avengers: Age of Ultron, Avengers: Infinity W...","[17, 10, 3613, 1209, 31]"


In [76]:
recommended_movies.to_csv('Recommendations.csv', index=False)

# Serializaing some important functions

In [77]:
def recommendationSystem(movie):
    res = []
    movie_index = movies_df[movies_df['Movie Name'] == movie].index[0]
    distances = similarity[movie_index]
    recommended_movies_index = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_movies, recommended_movies_id = [], []
    for i in recommended_movies_index:
        recommended_movies_id.append(i[0])
        recommended_movies.append(movies_df.iloc[i[0]]['Movie Name'])
    res = [recommended_movies_id, recommended_movies]
    return res

In [78]:
print(recommendationSystem(movies_df['Movie Name'][5]))

[[2597, 1448, 3734, 2164, 2214], ['Gotti', 'The Old Man & the Gun', 'Mr. Nice', 'The Beach Bum', 'True Crimes']]


In [79]:
pickle.dump(recommendationSystem, open('./Pickle Files/RecommendationSystem.pkl', 'wb'), fix_imports=True)

## Trying out the pickled function

In [80]:
recommendations = pickle.load(open('./Pickle Files/RecommendationSystem.pkl', 'rb'))

In [81]:
recommendations('Spider-Man: No Way Home')

[[131, 66, 28, 63, 1151],
 ['Spider-Man: Far from Home',
  'Spider-Man: Homecoming',
  'Iron Man 2',
  'The Amazing Spider-Man',
  'Pan']]